In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns

In [2]:
from sklearn.preprocessing import MinMaxScaler
print('Pandas : %s'%(pd.__version__))
print('Numpy : %s'%(np.__version__))
print('Scikit-Learn : %s'%(sklearn.__version__))
!python --version

Pandas : 1.0.2
Numpy : 1.18.1
Scikit-Learn : 0.22.1
Python 3.7.6


In [3]:
file = '2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터_v1_200818.xlsx'
# file = '2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx'
data = pd.read_excel(file, header=1, index_col = 0)
data.head()

,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
방송일시,,,,,,,
2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0


In [4]:
len(data['상품군'].value_counts())

12

In [5]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothic') # For Windows
print(plt.rcParams['font.family'])

['NanumGothic']


In [6]:
plt.style.use('seaborn-whitegrid')

### 평균적으로 판매된 단가보다 싸게 혹은 비싸게 판매되었는지 확인

In [7]:
def isthischeap(i):
    """
    같은 상품명으로 팔린 제품들의 평균 판매단가를 구하여
    평균단가보다 비싸게 팔린 경우 2,
    평균단가와 같은 가격에 팔린 경우 1,
    평균단가보다 저렴하게 팔린 경우 0 을 반환
    """
    name = data['상품명'][i]
    price = data['판매단가'][i]
    
    _df = data[data['상품명'] == name]    
    meanprice = _df['판매단가'].mean()
    
    if price < meanprice:
        return 0
    
    elif price > meanprice:
        return 2
    
    elif price == meanprice:
        return 1
    

In [8]:
mean_list = []
for i in range(len(data)):
    mean_list.append(isthischeap(i))

In [9]:
data['평균단가보다'] = mean_list
data

,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균단가보다
방송일시,,,,,,,,
2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,1
2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,1
2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,1
2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,1
2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,1
...,...,...,...,...,...,...,...,...
2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,1
2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,1
2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,1


In [10]:
## 2020년 데이터 삭제
data = data[:-9]
data

,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균단가보다
방송일시,,,,,,,,
2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,1
2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,1
2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,1
2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,1
2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,1
...,...,...,...,...,...,...,...,...
2019-12-31 23:20:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,1664000.0,1
2019-12-31 23:40:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,9149000.0,1
2019-12-31 23:40:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,15282000.0,1


### 상품명에서 브랜드를 추출

In [11]:
def Brandset(group):
    """
    상품명에서 브랜드를 추출하는 함수
    대부분 상품명 첫 번째 단어에 브랜드가 위치해있다는 특징을 활용
    예외는 수작업으로 전처리
    """
    name_list= []
    df = data[data['상품군'] == group]
    for words in df['상품명']:
        name_list.append(words)

    brand_set= []
    for name in name_list:
        brand_set.append(name.split()[0])
    brand_set = set(brand_set)

    return brand_set

In [12]:
# data 원본 별도 저장
df = data

#### 전체적 전처리

In [13]:
# 괄호
data.상품명 = data.상품명.str.replace('[', ' ', regex = False)
data.상품명 = data.상품명.str.replace(']', ' ', regex = False)
data.상품명 = data.상품명.str.replace('(', ' ', regex = False)
data.상품명 = data.상품명.str.replace(')', ' ', regex = False)

## 초특가
data.상품명 = data.상품명.str.replace('초특가', '')
data.상품명 = data.상품명.str.replace('일시불', '') 
data.상품명 = data.상품명.str.replace('무이자', '')

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


#### 주방 

In [14]:
## 연정쓰 전처리

data.상품명 = data.상품명.str.replace('쿠첸', ' 쿠첸 ')
data.상품명 = data.상품명.str.replace('쿠쿠', ' 쿠쿠 ')

# 쿠쿠전기밥솥
data.상품명 = data.상품명.str.replace('도냄', '', regex = False)
data.상품명 = data.상품명.str.replace('CRP-QS107FG/FS', '', regex = False)
data.상품명 = data.상품명.str.replace('CRP-107FG/FS', '', regex = False)
data.상품명 = data.상품명.str.replace('QS', '', regex = False) 
data.상품명 = data.상품명.str.replace('블랙스톤', '', regex = False) 

# 쿠첸 압력밥솥
data.상품명 = data.상품명.str.replace('풀스텐', '', regex = False)
data.상품명 = data.상품명.str.replace('A1', '', regex = False)
data.상품명 = data.상품명.str.replace(' 쿠 ', '', regex = False)
data.상품명 = data.상품명.str.replace('신제품', '', regex = False)

# 린나이 가스레인지
data.상품명 = data.상품명.str.replace('자동불꽃조절', '')
data.상품명 = data.상품명.str.replace('간편쿡', '')
data.상품명 = data.상품명.str.replace('2구', '')
data.상품명 = data.상품명.str.replace('3구', '')

# 에코라믹
data.상품명 = data.상품명.str.replace('냄비세트', ' 냄비 세트')
data.상품명 = data.상품명.str.replace('프라이팬세트', ' 프라이팬 세트')
data.상품명 = data.상품명.str.replace('통주물', ' ')
data.상품명 = data.상품명.str.replace('스톤', ' ')

# 후라이팬
data.상품명 = data.상품명.str.replace('후라이팬', ' 프라이팬 ')

# 에어프라이어
data.상품명 = data.상품명.str.replace('대용량', '')
data.상품명 = data.상품명.str.replace('디지털', '')
data.상품명 = data.상품명.str.replace('듀얼쿡', '')


data.상품명 = data.상품명.str.replace('가스와이드그릴레인지', '가스와이드그릴')
data.상품명 = data.상품명.str.replace('스텐큐브', '')
data.상품명 = data.상품명.str.replace('멀티쿡', '멀티쿠커')
data.상품명 = data.상품명.str.replace('팬지', '')

# 도마
data.상품명 = data.상품명.str.replace('도마', ' 도마 ')

data.상품명 = data.상품명.str.replace('대용량', '')

# 재질
data.상품명 = data.상품명.str.replace('스텐', ' 스텐 ')
data.상품명 = data.상품명.str.replace('유리', ' 유리 ')

data.상품명 = data.상품명.str.replace('밀폐용기', ' 밀폐용기')
data.상품명 = data.상품명.str.replace('플랫타입', '')

data.상품명 = data.상품명.str.replace('멀티', '')

data.상품명 = data.상품명.str.replace('만능', '')

def kitchen(x):
    if x.상품군 == '주방':
        x.상품명 = x.상품명.replace('옛', '')
        x.상품명 = x.상품명.replace('멀티', '')
        x.상품명 = x.상품명.replace('크로커다일', '')
    else:
        x.상품명 = x.상품명
    return x

data = data.apply(lambda x: kitchen(x), axis = 1)

data.상품명 = data.상품명.str.replace(' +', ' ')
data.상품명 = data.상품명.str.lstrip()
data.상품명 = data.상품명.str.rstrip()

In [15]:
data.상품명 = data.상품명.str.replace('(무)', '무이자', regex = False)
data.상품명 = data.상품명.str.replace('무)', '무이자', regex = False)
data.상품명 = data.상품명.str.replace('무이자', ' 무이자 ', regex = False)
data.상품명 = data.상품명.str.replace('(일)', '일시불', regex = False)
data.상품명 = data.상품명.str.replace('일)', '일시불', regex = False)
data.상품명 = data.상품명.str.replace('일시불', ' 일시불 ', regex = False)
data.상품명 = data.상품명.str.replace('해피콜', '해피콜 ', regex = False)
data.상품명 = data.상품명.str.replace('am', 'am ', regex = False)
data.상품명 = data.상품명.str.replace('베스트 하임', '베스트하임', regex = False)
data.상품명 = data.상품명.str.replace('쿠진', '쿠진 ', regex = False)
data.상품명 = data.상품명.str.replace('국내생산', '', regex = False)
data.상품명 = data.상품명.str.replace('국내제조', '', regex = False)
data.상품명 = data.상품명.str.replace('프랑스직수입', '', regex = False)
data.상품명 = data.상품명.str.replace('서장훈', '서장훈 ', regex = False)
data.상품명 = data.상품명.str.replace('휴롬', '휴롬 ', regex = False)
data.상품명 = data.상품명.str.replace('형', '', regex = False)
data.상품명 = data.상품명.str.replace('19년', '')
data.상품명 = data.상품명.str.replace('1세트', '')
data.상품명 = data.상품명.str.replace('2019년', '')
data.상품명 = data.상품명.str.replace('2세트', '')
data.상품명 = data.상품명.str.replace('20', '')

In [16]:
setKitchen = Brandset('주방')
setKitchen

{'IH',
 'PN풍년꾸노죽제조기',
 'SK매직',
 'am',
 '구스터',
 '글라스락',
 '노와',
 '뉴',
 '뉴욕맘',
 '델첸',
 '도깨비그릴',
 '두꺼비',
 '라니',
 '락앤락',
 '램프쿡',
 '로벤탈',
 '리큅',
 '린나이',
 '마이베비',
 '매직쉐프',
 '모즈',
 '믹서를',
 '베스트하임',
 '벨라홈',
 '서장훈',
 '세균싹',
 '세라맥스',
 '센스락무선진공포장기',
 '셀렉프로',
 '셰프웨이',
 '송도순의',
 '스마트코너',
 '스위스밀리터리',
 '스텐',
 '실리만',
 '실바트',
 '쓰임',
 '아이넥스',
 '아이오',
 '안타고',
 '에델코첸',
 '에버홈',
 '에지리',
 '에코라믹',
 '에코바이런',
 '오스터',
 '오슬로',
 '옥샘쿡',
 '올리고',
 '웰스락',
 '이지엔',
 '전기식',
 '쿠진',
 '쿠첸',
 '쿠쿠',
 '클란츠',
 '키친아트',
 '키친플라워',
 '테팔',
 '파뷔에',
 '프로피쿡',
 '하우홈',
 '한샘',
 '한일',
 '해피콜',
 '휴롬'}

In [17]:
setKitchen.remove('뉴')
setKitchen.remove('전기식')
setKitchen.add('클레린')
setKitchen.add('드럼쿡')
setKitchen

{'IH',
 'PN풍년꾸노죽제조기',
 'SK매직',
 'am',
 '구스터',
 '글라스락',
 '노와',
 '뉴욕맘',
 '델첸',
 '도깨비그릴',
 '두꺼비',
 '드럼쿡',
 '라니',
 '락앤락',
 '램프쿡',
 '로벤탈',
 '리큅',
 '린나이',
 '마이베비',
 '매직쉐프',
 '모즈',
 '믹서를',
 '베스트하임',
 '벨라홈',
 '서장훈',
 '세균싹',
 '세라맥스',
 '센스락무선진공포장기',
 '셀렉프로',
 '셰프웨이',
 '송도순의',
 '스마트코너',
 '스위스밀리터리',
 '스텐',
 '실리만',
 '실바트',
 '쓰임',
 '아이넥스',
 '아이오',
 '안타고',
 '에델코첸',
 '에버홈',
 '에지리',
 '에코라믹',
 '에코바이런',
 '오스터',
 '오슬로',
 '옥샘쿡',
 '올리고',
 '웰스락',
 '이지엔',
 '쿠진',
 '쿠첸',
 '쿠쿠',
 '클란츠',
 '클레린',
 '키친아트',
 '키친플라워',
 '테팔',
 '파뷔에',
 '프로피쿡',
 '하우홈',
 '한샘',
 '한일',
 '해피콜',
 '휴롬'}

#### 가전

In [18]:
data.상품명 = data.상품명.str.replace('1등급', '')
data.상품명 = data.상품명.str.replace('221L_', '')
data.상품명 = data.상품명.str.replace('467L_', '')
data.상품명 = data.상품명.str.replace('국내제조', '')
data.상품명 = data.상품명.str.replace('LG', 'LG ')
data.상품명 = data.상품명.str.replace('삼성', '삼성 ')
data.상품명 = data.상품명.str.replace('딤채', '딤채 ')

In [19]:
setDigital = Brandset('가전')
setDigital

{'LG', '대우전자', '딤채', '삼성', '캐리어'}

#### 의류

In [20]:
def clothes(x):
    if x.상품군 == '의류':
        x.상품명 = x.상품명.replace('SS', 'S/S')
        x.상품명 = x.상품명.replace('K-SWIS/S', 'K-SWISS')
        x.상품명 = x.상품명.replace('FW', 'F/W')
        x.상품명 = x.상품명.replace('S/S', ' S/S ')
        x.상품명 = x.상품명.replace('F/W', ' F/W ')

        x.상품명 = x.상품명.replace('기모', ' 기모 ')
        x.상품명 = x.상품명.replace('니트', ' 니트 ')
        x.상품명 = x.상품명.replace('팬츠', ' 팬츠 ')        
        x.상품명 = x.상품명.replace('데님 팬츠', ' 데님 ')
        x.상품명 = x.상품명.replace('티셔츠', ' 티셔츠 ')
        x.상품명 = x.상품명.replace('밴딩', ' 밴딩 ')

        x.상품명 = x.상품명.replace('코트', ' 코트 ')
        x.상품명 = x.상품명.replace('밍크', ' 밍크 ')
        x.상품명 = x.상품명.replace('그레이스', ' 그레이스 ')
    else:
        x.상품명 = x.상품명
    return x

data = data.apply(lambda x: clothes(x), axis = 1)

data.상품명 = data.상품명.str.replace('CERINI by PAT', 'CERINI_by_PAT') 
data.상품명 = data.상품명.str.replace('USPA', 'USPA ')

## 마르엘라로사티
data.상품명 = data.상품명.str.replace('풀스킨', '')
data.상품명 = data.상품명.str.replace('휘메일', '') #?

data.상품명 = data.상품명.str.replace('트랙수트', ' 트랙수트 ')

data.상품명 = data.상품명.str.replace('19', '')
data.상품명 = data.상품명.str.replace('2019', '')
data.상품명 = data.상품명.str.replace('20', '')
data.상품명 = data.상품명.str.replace('F/W', '')
data.상품명 = data.상품명.str.replace('S/S', '')

In [21]:
setClothes = Brandset('의류')
setClothes

{'CERINI_by_PAT',
 'EXR',
 'K-SWISS',
 'NNF',
 'USPA',
 '그렉노먼',
 '대동모피',
 '더블유베일',
 '도네이',
 '디베이지',
 '디즈니',
 '디키즈',
 '라라쎄',
 '레드캠프',
 '로이몬스터',
 '루이바셋',
 '르까프',
 '릴리젼',
 '마담팰리스',
 '마르엘라로사티',
 '마리노블',
 '마모트',
 '메시제이',
 '메이듀',
 '뱅뱅',
 '보코',
 '스텔라테일러',
 '스튜디오럭스',
 '아리스토우',
 '아문센',
 '아주아',
 '알렉스하운드',
 '어반시크릿',
 '에르나벨',
 '엔셀라두스',
 '오렐리안',
 '유리',
 '이동수골프',
 '임페리얼',
 '젠트웰',
 '코몽트',
 '코펜하겐럭스',
 '크리스티나앤코',
 '타운젠트',
 '테이트',
 '팜스프링스',
 '페플럼제이',
 '헤비추얼',
 '헤스티지'}

#### 속옷

In [22]:
data.상품명 = data.상품명.str.replace('ARS10%', '')
data.상품명 = data.상품명.str.replace('세일%', '')
data.상품명 = data.상품명.str.replace('완벽더블', '')
data.상품명 = data.상품명.str.replace('라쉬반', '라쉬반 ')

In [23]:
setInner = Brandset('속옷')
setInner

{'BBC&IZUMI',
 'LSX라이크라',
 '남영비비안',
 '댄스킨',
 '라쉬반',
 '란체티',
 '레이프릴',
 '로베르타',
 '루시헨느',
 '리복',
 '몬테밀라노',
 '발레리',
 '벨레즈온',
 '뷰티플렉스',
 '실크트리',
 '실크플러스',
 '아키',
 '에버라스트',
 '오가닉뷰티',
 '오렐리안',
 '오모떼',
 '저스트마이사이즈',
 '카파',
 '컬럼비아',
 '코몽트',
 '쿠미투니카',
 '크로커다일',
 '푸마',
 '헤드',
 '헤스떼벨',
 '히트융'}

#### 잡화

In [24]:
data.상품명 = data.상품명.str.replace('무', '')
data.상품명 = data.상품명.str.replace('일', '')

In [25]:
setStuff = Brandset('잡화')
setStuff

{'AAA',
 'AAD',
 'DIOR',
 'RYN',
 '가이거',
 '갈란테',
 '골드파',
 '구찌',
 '기라로쉬',
 '도스문도스',
 '레노마',
 '레코바',
 '로베르타',
 '루이띠에',
 '마스케라',
 '마이클코어스',
 '메디아글램',
 '메이듀',
 '바치',
 '버버리',
 '삭루츠',
 '생줄랑',
 '생쥴랑',
 '세인트스코트',
 '스프리스',
 '시스마르스',
 '썸덱스',
 '아가타',
 '아르테사노',
 '안드레아바나',
 '알비에로',
 '에버라스트',
 '에트로',
 '에펨',
 '엘르',
 '엘리자베스아덴',
 '오델로',
 '월드컵',
 '제옥스',
 '칼리베이직',
 '코치',
 '트레스패스',
 '프라다',
 '플로쥬'}

#### 생활용품

In [26]:
data.상품명 = data.상품명.str.replace('대형', '', regex = False)
data.상품명 = data.상품명.str.replace('더블+더블', '', regex = False)
data.상품명 = data.상품명.str.replace('더블+싱글', '', regex = False)
data.상품명 = data.상품명.str.replace('더블사이즈', '', regex = False)
data.상품명 = data.상품명.str.replace('싱글사이즈', '', regex = False)
data.상품명 = data.상품명.str.replace('싱글+싱글', '', regex = False)
data.상품명 = data.상품명.str.replace('점보', '', regex = False)
data.상품명 = data.상품명.str.replace('점보특대형', '', regex = False)
data.상품명 = data.상품명.str.replace('점보형', '', regex = False)
data.상품명 = data.상품명.str.replace('중형', '', regex = False)
data.상품명 = data.상품명.str.replace('퀸+싱글', '', regex = False)
data.상품명 = data.상품명.str.replace('킹+싱글', '', regex = False)
data.상품명 = data.상품명.str.replace('퀸+퀸', '', regex = False)
data.상품명 = data.상품명.str.replace('퀸사이즈', '', regex = False)
data.상품명 = data.상품명.str.replace('퀸+싱글', '', regex = False)
data.상품명 = data.상품명.str.replace('킹사이즈', '', regex = False)
data.상품명 = data.상품명.str.replace('특대', '', regex = False)
data.상품명 = data.상품명.str.replace('1+1', '', regex = False)
data.상품명 = data.상품명.str.replace('3D매쉬', '', regex = False)
data.상품명 = data.상품명.str.replace('1세트', '', regex = False)
data.상품명 = data.상품명.str.replace('5세트', '', regex = False)
data.상품명 = data.상품명.str.replace('ALL', '', regex = False)
data.상품명 = data.상품명.str.replace('D)', '', regex = False)
data.상품명 = data.상품명.str.replace('OK', '', regex = False)
data.상품명 = data.상품명.str.replace('국내제작', '', regex = False)
data.상품명 = data.상품명.str.replace('국내제조', '', regex = False)
data.상품명 = data.상품명.str.replace('기본구성', '', regex = False)
data.상품명 = data.상품명.str.replace('파격가', '', regex = False)
data.상품명 = data.상품명.str.replace('초특가', '', regex = False)
data.상품명 = data.상품명.str.replace('[실속패키지]', '', regex = False)
data.상품명 = data.상품명.str.replace('[풀패키지]', '', regex = False)
data.상품명 = data.상품명.str.replace('12.5m x 3롤', '', regex = False)
data.상품명 = data.상품명.str.replace('12.5m x 2롤', '', regex = False)
data.상품명 = data.상품명.str.replace('12.5m x 1롤', '', regex = False)
data.상품명 = data.상품명.str.replace('2세트', '', regex = False)
data.상품명 = data.상품명.str.replace('KF94', '', regex = False)
data.상품명 = data.상품명.str.replace('엔웰스', '엔웰스 ', regex = False)
data.상품명 = data.상품명.str.replace('D', '', regex = False)
data.상품명 = data.상품명.str.replace('New', '', regex = False)
data.상품명 = data.상품명.str.replace('년', '', regex = False)
data.상품명 = data.상품명.str.replace('올바로', '올바로 ', regex = False)
data.상품명 = data.상품명.str.replace('까사마루', '까사마루 ', regex = False)
data.상품명 = data.상품명.str.replace('붙이는', '', regex = False)
data.상품명 = data.상품명.str.replace('레스트업', '', regex = False)
data.상품명 = data.상품명.str.replace('가 ', '', regex = False)
data.상품명 = data.상품명.str.replace('대 ', '', regex = False)
data.상품명 = data.상품명.str.replace('올뉴', '', regex = False)
data.상품명 = data.상품명.str.replace('실속패키지', '', regex = False)
data.상품명 = data.상품명.str.replace('풀패키지', '', regex = False)

In [27]:
setLife = Brandset('생활용품')  # 일월, 한일 추가
setLife

{'LG',
 '거미손',
 '구성',
 '굿프렌드',
 '근육통완화',
 '김병만의',
 '김병지',
 '까사마루',
 '냉수마찰',
 '노비타',
 '노송가구',
 '니봇',
 '대웅모닝컴',
 '도루코',
 '디비노',
 '따스림',
 '루미테라피',
 '메디쉴드',
 '메디컬드림',
 '모리츠',
 '바두기',
 '바로바로',
 '바로톡',
 '발렌티노루디',
 '배관청소용품',
 '밸런스파워',
 '벨라홈',
 '보국미니히터',
 '브람스안마의자',
 '블루콤',
 '사운드룩',
 '선금고',
 '세렌셉템버',
 '센스톰알파청소기',
 '센스하우스',
 '센티멘탈',
 '숀리',
 '수련',
 '수액패치',
 '스위스밀리터리',
 '스칸디나비아',
 '스팀큐다리미',
 '스피드랙',
 '씨엔지코리아',
 '얼리젼',
 '엑사이더',
 '엔웰스',
 '올바로',
 '월',
 '월시스',
 '이지스',
 '자미코코',
 '중',
 '중외신약',
 '캐치온',
 '코이모',
 '코지마',
 '코튼데이',
 '퀸메이드',
 '크린조이',
 '킹스스파',
 '타이거',
 '테팔',
 '트라이',
 '파로마',
 '파워스윙',
 '페르소나',
 '한',
 '한세트',
 '한솔',
 '히팅맘'}

In [28]:
setLife.remove('중')
setLife.remove('월')
setLife.remove('한')
setLife.remove('LG')
setLife.add('LG생활건강')
setLife.add('일월')
setLife.add('한일')
setLife

{'LG생활건강',
 '거미손',
 '구성',
 '굿프렌드',
 '근육통완화',
 '김병만의',
 '김병지',
 '까사마루',
 '냉수마찰',
 '노비타',
 '노송가구',
 '니봇',
 '대웅모닝컴',
 '도루코',
 '디비노',
 '따스림',
 '루미테라피',
 '메디쉴드',
 '메디컬드림',
 '모리츠',
 '바두기',
 '바로바로',
 '바로톡',
 '발렌티노루디',
 '배관청소용품',
 '밸런스파워',
 '벨라홈',
 '보국미니히터',
 '브람스안마의자',
 '블루콤',
 '사운드룩',
 '선금고',
 '세렌셉템버',
 '센스톰알파청소기',
 '센스하우스',
 '센티멘탈',
 '숀리',
 '수련',
 '수액패치',
 '스위스밀리터리',
 '스칸디나비아',
 '스팀큐다리미',
 '스피드랙',
 '씨엔지코리아',
 '얼리젼',
 '엑사이더',
 '엔웰스',
 '올바로',
 '월시스',
 '이지스',
 '일월',
 '자미코코',
 '중외신약',
 '캐치온',
 '코이모',
 '코지마',
 '코튼데이',
 '퀸메이드',
 '크린조이',
 '킹스스파',
 '타이거',
 '테팔',
 '트라이',
 '파로마',
 '파워스윙',
 '페르소나',
 '한세트',
 '한솔',
 '한일',
 '히팅맘'}

#### 가구

In [29]:
data.상품명 = data.상품명.str.replace('3인용', '', regex = False)
data.상품명 = data.상품명.str.replace('4인용', '', regex = False)
data.상품명 = data.상품명.str.replace('장수', '장수 ', regex = False)
data.상품명 = data.상품명.str.replace('SET', '', regex = False)
data.상품명 = data.상품명.str.replace('뉴', '', regex = False)

In [30]:
setFurniture = Brandset('가구')
setFurniture

{'레스토닉', '벨라홈', '보루네오', '삼익가구', '유캐슬', '이누스바스', '이조농방', '장수', '한샘'}

#### 이미용

In [31]:
data.상품명 = data.상품명.str.replace('NEW프리미엄', '', regex = False)
data.상품명 = data.상품명.str.replace('TS', 'TS ', regex = False)

In [32]:
setBeauty = Brandset('이미용')
setBeauty.remove('시크릿')
setBeauty.add('라메종')
setBeauty.remove('프리미엄')
setBeauty.add('클린샤워')
setBeauty

{'AHC',
 'TS',
 'VONIN',
 '고데롤',
 '네오젠',
 '달바',
 '더블모',
 '라라츄',
 '라메종',
 '마리끌레르',
 '메디앤서',
 '미바',
 '바바코코',
 '블링썸',
 '비버리힐스폴로클럽',
 '살롱클리닉',
 '스칼프솔루션',
 '스포메틱스',
 '시크릿뮤즈',
 '실크테라피',
 '아미니',
 '아이앤아이',
 '에이온에이',
 '에이유플러스',
 '엘렌실라',
 '엘렌실라&코즈미',
 '엘로엘',
 '웨이',
 '자올',
 '참존',
 '컨시크',
 '코튼플러스',
 '클린샤워',
 '파시노',
 '피부약방'}

#### 건강기능

In [33]:
data.상품명 = data.상품명.str.replace('"', '', regex = False)
data.상품명 = data.상품명.str.replace('농약', '', regex = False)
data.상품명 = data.상품명.str.replace('단하루', '', regex = False)
data.상품명 = data.상품명.str.replace('직매입', '', regex = False)
data.상품명 = data.상품명.str.replace('특집', '', regex = False)
data.상품명 = data.상품명.str.replace('100%리얼', '100%리얼 ', regex = False)
data.상품명 = data.상품명.str.replace('한삼인', '한삼인 ', regex = False)

In [34]:
setHealth = Brandset('건강기능')
setHealth.remove('한국인')
setHealth.add('비에날씬')
setHealth

{'100%리얼',
 '경남제약',
 '광동',
 '광동제약',
 '네페르티티',
 '닥터',
 '레드비트즙',
 '리얼니파팜',
 '모나코사놀',
 '밀싹',
 '베지밀',
 '블랙모어스',
 '비에날씬',
 '서울더',
 '안국루테인',
 '여자를',
 '이경제의',
 '이롬',
 '정직한',
 '제주농장',
 '종근당건강',
 '질랜드',
 '콜라겐3.2',
 '통째로',
 '트리원',
 '티젠',
 '프리바이오틱스',
 '한삼인'}

#### 침구

In [35]:
setBed = Brandset('침구')
setBed.remove('리앤코리아')
setBed

{'리앤', '보몽드', '안지', '한빛', '한샘', '효재'}

#### 농수축

In [36]:
Brandset('농수축')

{'AAB의',
 'AAC',
 'AAC한우소머리곰탕양곰탕세트',
 'AAE',
 'SPC삼립',
 'a4',
 '가격인하',
 '강레오의',
 '강원도양구',
 '거창특등급사과',
 '고창',
 '고칼슘검은콩두유48팩+호두아몬드',
 '구워만든',
 '국내산',
 '국내산참조기12팩',
 '궁중',
 '김선영',
 '김정문의',
 '김정배',
 '깐깐송도순',
 '남해바다',
 '농협',
 '농협안심벌꿀',
 '더커진거창특등급사과',
 '맛있는',
 '멋진밥상',
 '명인',
 '목우촌',
 '바다먹자',
 '바다원',
 '본죽',
 '뽕셰프',
 '사용불가미리구운',
 '산머루농원',
 '소들녘',
 '속초명물',
 '수협',
 '슬로푸드',
 '신세포기김치',
 '안동간고등어',
 '영광',
 '영광군수협',
 '영산포숙성',
 '영산포숙성홍어회7팩',
 '예천청결',
 '옛날',
 '오세득',
 '완도꼬마활전복',
 '완도산',
 '우리나라',
 '우리바다',
 '유귀열의',
 '이경제원장의',
 '이만기의',
 '이보은의',
 '이봉원',
 '이정섭의',
 '임성근의',
 '자연산',
 '자이언트킹랍스터',
 '잔다리',
 '장보고',
 '전철우',
 '제주',
 '제주갈치+참조기세트',
 '제주갈치고등어구이세트',
 '제주바다',
 '조방할매',
 '참바다손질낙지+양념장+연포탕육수',
 '참바다손질낙지100미+양념장+연포탕육수',
 '창녕',
 '천수봉명인',
 '천연담아',
 '청정수산',
 '최인선',
 '캘리포니아',
 '통뼈',
 '통영바다의보물',
 '특등급',
 '포항구룡포과메기세트',
 '피시원',
 '하늘내린',
 '하동',
 '하림',
 '하림뼈없는양념닭발세트',
 '현대어찬',
 '황토방'}

In [37]:
food = ['팽현숙', '전철우', '농협', '바다원', '본죽', '수협', '강레오', 'SPC 삼립',
        '김선영', '김정문', '김정배', '송도순', '명인', '목우촌', '바다먹자',
        '뽕셰프', '산머루농원', '소들녘', '오세득', '유귀열', '이경제', '이만기',
        '이보은', '이봉원', '이정섭', '임성근', '장보고', '조방할매', '참바다',
        '천수봉명인', '최인선', '피시원', '하늘내린', '하동', '하림', '현대어찬', '황토방']

### 데이터프레임에 합치기

In [38]:
## 리스트로 변환
kitchen = list(setKitchen)
digital = list(setDigital)
clothes = list(setClothes)
inner = list(setInner)
stuff = list(setStuff)
life = list(setLife)
furniture = list(setFurniture)
beauty = list(setBeauty)
health = list(setHealth)
bed = list(setBed)

In [39]:
## 의류에서 띄어쓰기가 포함된 브랜드 처리
clothes.remove('CERINI_by_PAT')
clothes.append('CERINI by PAT')

In [40]:
def Brand(group, name):
    for brand in group:
        if brand in name:
            return brand

In [41]:
brand_list = []
for i in range(len(data)):
    group = data['상품군'].iloc[i]
    name = data['상품명'].iloc[i]
    
    if group == '주방':
        brand_list.append(Brand(kitchen, name))
    elif group == '가전':
        brand_list.append(Brand(digital, name))
    elif group == '의류':
        brand_list.append(Brand(clothes, name))
    elif group == '속옷':
        brand_list.append(Brand(inner, name))
    elif group == '잡화':
        brand_list.append(Brand(stuff, name))
    elif group == '생활용품':
        brand_list.append(Brand(life, name))
    elif group == '가구':
        brand_list.append(Brand(furniture, name))
    elif group == '이미용':
        brand_list.append(Brand(beauty, name))
    elif group == '건강기능':
        brand_list.append(Brand(health, name))
    elif group == '침구':
        brand_list.append(Brand(bed, name))
    elif group == '농수축':
        brand_list.append(Brand(food, name))
    else:
        brand_list.append('무형')

In [42]:
df['브랜드'] = brand_list
df

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균단가보다,브랜드
방송일시,,,,,,,,,
2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,1,테이트
2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,1,테이트
2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,1,테이트
2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,1,테이트
2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,1,테이트
...,...,...,...,...,...,...,...,...,...
2019-12-31 23:20:00,NaN,100448,201391,쿠첸 압력밥솥 6인용,주방,148000,1664000.0,1,쿠첸
2019-12-31 23:40:00,20.0,100448,201383,쿠첸 압력밥솥 10인용,주방,178000,9149000.0,1,쿠첸
2019-12-31 23:40:00,NaN,100448,201390,쿠첸 압력밥솥 10인용,주방,168000,15282000.0,1,쿠첸


### 시간정보

In [43]:
data = df
data

,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균단가보다,브랜드
방송일시,,,,,,,,,
2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,1,테이트
2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,1,테이트
2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,1,테이트
2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,1,테이트
2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,1,테이트
...,...,...,...,...,...,...,...,...,...
2019-12-31 23:20:00,NaN,100448,201391,쿠첸 압력밥솥 6인용,주방,148000,1664000.0,1,쿠첸
2019-12-31 23:40:00,20.0,100448,201383,쿠첸 압력밥솥 10인용,주방,178000,9149000.0,1,쿠첸
2019-12-31 23:40:00,NaN,100448,201390,쿠첸 압력밥솥 10인용,주방,168000,15282000.0,1,쿠첸


In [44]:
date = pd.to_datetime(data.index)

data['판매횟수'] = data['취급액']/data['판매단가']
data['방송월'] = date.strftime('%m')
data['방송일'] = date.strftime('%d')
data['방송시간'] = date.strftime('%H:%M')
data['방송시'] = date.strftime('%H')
data['방송분'] = date.strftime('%M')
data['방송요일'] = date.strftime('%a')

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [45]:
data

,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균단가보다,브랜드,판매횟수,방송월,방송일,방송시간,방송시,방송분,방송요일
방송일시,,,,,,,,,,,,,,,,
2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,1,테이트,52.606516,01,01,06:00,06,00,Tue
2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,1,테이트,109.548872,01,01,06:00,06,00,Tue
2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,1,테이트,81.754386,01,01,06:20,06,20,Tue
2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,1,테이트,174.310777,01,01,06:20,06,20,Tue
2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,1,테이트,167.218045,01,01,06:40,06,40,Tue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 23:20:00,NaN,100448,201391,쿠첸 압력밥솥 6인용,주방,148000,1664000.0,1,쿠첸,11.243243,12,31,23:20,23,20,Tue
2019-12-31 23:40:00,20.0,100448,201383,쿠첸 압력밥솥 10인용,주방,178000,9149000.0,1,쿠첸,51.398876,12,31,23:40,23,40,Tue
2019-12-31 23:40:00,NaN,100448,201390,쿠첸 압력밥솥 10인용,주방,168000,15282000.0,1,쿠첸,90.964286,12,31,23:40,23,40,Tue


In [46]:
def get_prime(x):
    prime_bn = map(str,[8, 9, 10, 11])
    prime_an = map(str,[20,21, 22, 23])
    
    x = x
    #print(x)
    if x in prime_bn:
        return 1
    else:
        return 0

data['프라임'] = data['방송시'].map(lambda x: get_prime(x))

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [47]:
data['정각'] = data['방송분'].map(lambda x: 1 if x == '00' else 0)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
## 타겟 데이터는 공휴일 없음!
def get_holiday(x):
    date = str(x.방송월) +'-'+ str(x.방송일)
    holiday = ['01-01', '02-04', '02-05', '02-06', '03-01', '05-05', '05-06', '05-12',
               '06-06', '08-15', '09-12', '09-13', '09-14', '10-03', '10-09', '12-25']
    if date in holiday :
        return 1
    else:
        return 0

data['공휴일'] = data.apply(lambda x: get_holiday(x), axis = 1)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [49]:
data['주말'] = data.apply(lambda x : 1 if x.방송요일 == 'Sat' or x.방송요일 == 'Sun' else 0, axis = 1)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [50]:
data

,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균단가보다,브랜드,판매횟수,방송월,방송일,방송시간,방송시,방송분,방송요일,프라임,정각,공휴일,주말
방송일시,,,,,,,,,,,,,,,,,,,,
2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,1,테이트,52.606516,01,01,06:00,06,00,Tue,0,1,1,0
2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,1,테이트,109.548872,01,01,06:00,06,00,Tue,0,1,1,0
2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,1,테이트,81.754386,01,01,06:20,06,20,Tue,0,0,1,0
2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,1,테이트,174.310777,01,01,06:20,06,20,Tue,0,0,1,0
2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,1,테이트,167.218045,01,01,06:40,06,40,Tue,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 23:20:00,NaN,100448,201391,쿠첸 압력밥솥 6인용,주방,148000,1664000.0,1,쿠첸,11.243243,12,31,23:20,23,20,Tue,0,0,0,0
2019-12-31 23:40:00,20.0,100448,201383,쿠첸 압력밥솥 10인용,주방,178000,9149000.0,1,쿠첸,51.398876,12,31,23:40,23,40,Tue,0,0,0,0
2019-12-31 23:40:00,NaN,100448,201390,쿠첸 압력밥솥 10인용,주방,168000,15282000.0,1,쿠첸,90.964286,12,31,23:40,23,40,Tue,0,0,0,0


### 연속 편성 정보

In [51]:
expo_df = data.groupby(['방송월','방송일','상품명']).count()['상품군']
date_list = []
name_list = []

def exposure_num(x):
    date = str(x.방송월)+'/'+str(x.방송일)
    if date not in date_list:
        date_list.append(date)
        name_list.clear()

    name_list.append(x.상품명)
    
    return name_list.count(x.상품명)/(expo_df.loc[(x.방송월, x.방송일, x.상품명)])

data['파트'] = data.apply(lambda x: exposure_num(x), axis=1)
data

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균단가보다,브랜드,판매횟수,...,방송일,방송시간,방송시,방송분,방송요일,프라임,정각,공휴일,주말,파트
방송일시,,,,,,,,,,,,,,,,,,,,,
2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,1,테이트,52.606516,...,01,06:00,06,00,Tue,0,1,1,0,0.333333
2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,1,테이트,109.548872,...,01,06:00,06,00,Tue,0,1,1,0,0.333333
2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,1,테이트,81.754386,...,01,06:20,06,20,Tue,0,0,1,0,0.666667
2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,1,테이트,174.310777,...,01,06:20,06,20,Tue,0,0,1,0,0.666667
2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,1,테이트,167.218045,...,01,06:40,06,40,Tue,0,0,1,0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 23:20:00,NaN,100448,201391,쿠첸 압력밥솥 6인용,주방,148000,1664000.0,1,쿠첸,11.243243,...,31,23:20,23,20,Tue,0,0,0,0,0.500000
2019-12-31 23:40:00,20.0,100448,201383,쿠첸 압력밥솥 10인용,주방,178000,9149000.0,1,쿠첸,51.398876,...,31,23:40,23,40,Tue,0,0,0,0,0.750000
2019-12-31 23:40:00,NaN,100448,201390,쿠첸 압력밥솥 10인용,주방,168000,15282000.0,1,쿠첸,90.964286,...,31,23:40,23,40,Tue,0,0,0,0,1.000000


In [52]:
def part_(x):
    if x <= 0.5:
        return 'First'
    elif 0.5 < x < 1:
        return 'Middle'
    elif x == 1:
      return 'Later'

data['파트'] = data.파트.map(lambda x: part_(x))

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [53]:
data

,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균단가보다,브랜드,판매횟수,...,방송일,방송시간,방송시,방송분,방송요일,프라임,정각,공휴일,주말,파트
방송일시,,,,,,,,,,,,,,,,,,,,,
2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,1,테이트,52.606516,...,01,06:00,06,00,Tue,0,1,1,0,First
2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,1,테이트,109.548872,...,01,06:00,06,00,Tue,0,1,1,0,First
2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,1,테이트,81.754386,...,01,06:20,06,20,Tue,0,0,1,0,Middle
2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,1,테이트,174.310777,...,01,06:20,06,20,Tue,0,0,1,0,Middle
2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,1,테이트,167.218045,...,01,06:40,06,40,Tue,0,0,1,0,Later
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 23:20:00,NaN,100448,201391,쿠첸 압력밥솥 6인용,주방,148000,1664000.0,1,쿠첸,11.243243,...,31,23:20,23,20,Tue,0,0,0,0,First
2019-12-31 23:40:00,20.0,100448,201383,쿠첸 압력밥솥 10인용,주방,178000,9149000.0,1,쿠첸,51.398876,...,31,23:40,23,40,Tue,0,0,0,0,Middle
2019-12-31 23:40:00,NaN,100448,201390,쿠첸 압력밥솥 10인용,주방,168000,15282000.0,1,쿠첸,90.964286,...,31,23:40,23,40,Tue,0,0,0,0,Later


## Submission

In [55]:
file = '2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx'
data = pd.read_excel(file, header=1, index_col = 0)
data.head()

,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
방송일시,,,,,,,
2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN


In [56]:
def isthischeap(i):
    """
    같은 상품명으로 팔린 제품들의 평균 판매단가를 구하여
    평균단가보다 비싸게 팔린 경우 2,
    평균단가와 같은 가격에 팔린 경우 1,
    평균단가보다 저렴하게 팔린 경우 0 을 반환
    """
    name = data['상품명'][i]
    price = data['판매단가'][i]
    
    _df = data[data['상품명'] == name]    
    meanprice = _df['판매단가'].mean()
    
    if price < meanprice:
        return 0
    
    elif price > meanprice:
        return 2
    
    elif price == meanprice:
        return 1
    
mean_list = []
for i in range(len(data)):
    mean_list.append(isthischeap(i))
    
data['평균단가보다'] = mean_list
data

,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균단가보다
방송일시,,,,,,,,
2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1
2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1
2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1
2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,1
2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,1
...,...,...,...,...,...,...,...,...
2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,1
2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,1
2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,1


In [61]:
len(data)

2891

In [60]:
len(brand_list)

2891

In [62]:
brand_list = []
for i in range(len(data)):
    group = data['상품군'].iloc[i]
    name = data['상품명'].iloc[i]
    
    if group == '주방':
        brand_list.append(Brand(kitchen, name))
    elif group == '가전':
        brand_list.append(Brand(digital, name))
    elif group == '의류':
        brand_list.append(Brand(clothes, name))
    elif group == '속옷':
        brand_list.append(Brand(inner, name))
    elif group == '잡화':
        brand_list.append(Brand(stuff, name))
    elif group == '생활용품':
        brand_list.append(Brand(life, name))
    elif group == '가구':
        brand_list.append(Brand(furniture, name))
    elif group == '이미용':
        brand_list.append(Brand(beauty, name))
    elif group == '건강기능':
        brand_list.append(Brand(health, name))
    elif group == '침구':
        brand_list.append(Brand(bed, name))
    elif group == '농수축':
        brand_list.append(Brand(food, name))
    else:
        brand_list.append('무형')

In [64]:
data['브랜드'] = brand_list
data

,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균단가보다,판매횟수,방송월,방송일,방송시간,방송시,방송분,방송요일,브랜드
방송일시,,,,,,,,,,,,,,,,
2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1,NaN,06,01,06:20,06,20,Mon,None
2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1,NaN,06,01,06:40,06,40,Mon,None
2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1,NaN,06,01,07:00,07,00,Mon,None
2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,1,NaN,06,01,07:20,07,20,Mon,쿠미투니카
2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,1,NaN,06,01,07:40,07,40,Mon,쿠미투니카
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,1,NaN,07,01,00:20,00,20,Wed,무형
2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,1,NaN,07,01,00:40,00,40,Wed,무형
2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,1,NaN,07,01,01:00,01,00,Wed,무형


In [65]:
date = pd.to_datetime(data.index)

data['판매횟수'] = data['취급액']/data['판매단가']
data['방송월'] = date.strftime('%m')
data['방송일'] = date.strftime('%d')
data['방송시간'] = date.strftime('%H:%M')
data['방송시'] = date.strftime('%H')
data['방송분'] = date.strftime('%M')
data['방송요일'] = date.strftime('%a')

In [66]:
data['프라임'] = data['방송시'].map(lambda x: get_prime(x))

In [67]:
data['정각'] = data['방송분'].map(lambda x: 1 if x == '00' else 0)

In [68]:
data['공휴일'] = data.apply(lambda x: get_holiday(x), axis = 1)

In [69]:
data['주말'] = data.apply(lambda x : 1 if x.방송요일 == 'Sat' or x.방송요일 == 'Sun' else 0, axis = 1)

In [70]:
expo_df = data.groupby(['방송월','방송일','상품명']).count()['상품군']
date_list = []
name_list = []

In [71]:
data['파트'] = data.apply(lambda x: exposure_num(x), axis=1)
data

,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균단가보다,판매횟수,방송월,...,방송시간,방송시,방송분,방송요일,브랜드,프라임,정각,공휴일,주말,파트
방송일시,,,,,,,,,,,,,,,,,,,,,
2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1,NaN,06,...,06:20,06,20,Mon,None,0,0,0,0,0.333333
2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1,NaN,06,...,06:40,06,40,Mon,None,0,0,0,0,0.666667
2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1,NaN,06,...,07:00,07,00,Mon,None,0,1,0,0,1.000000
2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,1,NaN,06,...,07:20,07,20,Mon,쿠미투니카,0,0,0,0,0.333333
2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,1,NaN,06,...,07:40,07,40,Mon,쿠미투니카,0,0,0,0,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,1,NaN,07,...,00:20,00,20,Wed,무형,0,0,0,0,0.333333
2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,1,NaN,07,...,00:40,00,40,Wed,무형,0,0,0,0,0.666667
2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,1,NaN,07,...,01:00,01,00,Wed,무형,0,1,0,0,1.000000


In [72]:
data['파트'] = data.파트.map(lambda x: part_(x))

In [73]:
data

,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균단가보다,판매횟수,방송월,...,방송시간,방송시,방송분,방송요일,브랜드,프라임,정각,공휴일,주말,파트
방송일시,,,,,,,,,,,,,,,,,,,,,
2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1,NaN,06,...,06:20,06,20,Mon,None,0,0,0,0,First
2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1,NaN,06,...,06:40,06,40,Mon,None,0,0,0,0,Middle
2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1,NaN,06,...,07:00,07,00,Mon,None,0,1,0,0,Later
2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,1,NaN,06,...,07:20,07,20,Mon,쿠미투니카,0,0,0,0,First
2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,1,NaN,06,...,07:40,07,40,Mon,쿠미투니카,0,0,0,0,Middle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,1,NaN,07,...,00:20,00,20,Wed,무형,0,0,0,0,First
2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,1,NaN,07,...,00:40,00,40,Wed,무형,0,0,0,0,Middle
2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,1,NaN,07,...,01:00,01,00,Wed,무형,0,1,0,0,Later


In [74]:
data.to_csv('submission_preprocessing.csv')

In [76]:
data[data['브랜드'].isnull()]

,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균단가보다,판매횟수,방송월,...,방송시간,방송시,방송분,방송요일,브랜드,프라임,정각,공휴일,주말,파트
방송일시,,,,,,,,,,,,,,,,,,,,,
2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1,NaN,06,...,06:20,06,20,Mon,None,0,0,0,0,First
2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1,NaN,06,...,06:40,06,40,Mon,None,0,0,0,0,Middle
2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1,NaN,06,...,07:00,07,00,Mon,None,0,1,0,0,Later
2020-06-01 08:20:00,20.000000,100381,201247,바비리스 퍼펙트 볼륨스타일러,이미용,59000,NaN,1,NaN,06,...,08:20,08,20,Mon,None,0,0,0,0,First
2020-06-01 08:40:00,20.000000,100381,201247,바비리스 퍼펙트 볼륨스타일러,이미용,59000,NaN,1,NaN,06,...,08:40,08,40,Mon,None,0,0,0,0,Middle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-30 22:20:00,20.000000,100514,201578,해초미인슬림다시마국수세트25인분,농수축,39900,NaN,1,NaN,06,...,22:20,22,20,Tue,None,0,0,0,0,First
2020-06-30 22:40:00,20.000000,100514,201578,해초미인슬림다시마국수세트25인분,농수축,39900,NaN,1,NaN,06,...,22:40,22,40,Tue,None,0,0,0,0,Middle
2020-06-30 23:00:00,20.000000,100514,201578,해초미인슬림다시마국수세트25인분,농수축,39900,NaN,1,NaN,06,...,23:00,23,00,Tue,None,0,1,0,0,Later
